<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/rnn/genderClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NoCodeProgram/deepLearning.git

'deepLearning'에 복제합니다...
remote: Enumerating objects: 278, done.
remote: Counting objects: 100% (139/139), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 278 (delta 57), reused 134 (delta 55), pack-reused 139
오브젝트를 받는 중: 100% (278/278), 12.42 MiB | 16.88 MiB/s, 완료.
델타를 알아내는 중: 100% (93/93), 완료.


In [18]:
import pandas as pd


df = pd.read_csv('./deepLearning/rnn/name_gender_filtered.csv')
unique_chars = set()

for name in df['Name']:
    unique_chars.update(name)
    
unique_chars = sorted(list(unique_chars))
unique_chars = ''.join(unique_chars)
print(unique_chars)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
<class 'list'>
abcdefghijklmnopqrstuvwxyz
<class 'str'>


In [23]:
n_letters = len(unique_chars)
def nameToTensor(name):
    tensor = torch.zeros(len(name), n_letters)
    print(tensor)
    for char_idx, char in enumerate(name):
        letter_idx = unique_chars.find(char)
        assert letter_idx != -1, f"char is {name}, {char}"
        tensor[char_idx][letter_idx] = 1
    return tensor

gen2num = {'F':0, 'M':1}
num2gen = {0:'F', 1:'M'}

In [5]:
import torch.nn as nn
#Try other arch
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)


    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = torch.tanh(self.i2h(combined))
        output = self.h2o(hidden)
        return output, hidden

    def get_hidden(self):
        return torch.zeros(1, self.hidden_size)


n_hidden = 32
rnn_model = MyRNN(n_letters, n_hidden, 2)

In [13]:
import torch
from torch.optim import Adam, SGD

device = torch.device("cpu")
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")

print(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = Adam(rnn_model.parameters(), lr=0.0001)


rnn_model.train()
for epoch_idx in range(200):
    shuffled_df = df.sample(frac=1).reset_index(drop=True)

    total_loss = 0.
    correct_predictions = 0
    total_predictions = 0

    for index, row in shuffled_df.iterrows():
        input_tensor = nameToTensor(row['Name'])
        target_tensor = torch.tensor([gen2num[row['Gender']]], dtype=torch.long)

        hidden = rnn_model.get_hidden()

        rnn_model.zero_grad()

        for char_idx in range(input_tensor.size()[0]):
            char_tensor = input_tensor[char_idx]
            output, hidden = rnn_model(char_tensor[None,:], hidden)

        loss = loss_fn(output, target_tensor)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted_index = torch.argmax(output, 1)
        correct_predictions += (predicted_index == target_tensor).sum().item()
        total_predictions += 1

    average_loss = total_loss / total_predictions
    accuracy = 100 * correct_predictions / total_predictions
    print(f'Epoch: {epoch_idx}, Loss: {average_loss:.4f}, Accuracy: {accuracy:.2f}%')


mps
Epoch: 0, Loss: 0.3754, Accuracy: 83.72%
Epoch: 1, Loss: 0.3727, Accuracy: 83.78%
Epoch: 2, Loss: 0.3710, Accuracy: 83.86%
Epoch: 3, Loss: 0.3697, Accuracy: 83.87%


KeyboardInterrupt: 

In [24]:
test_name = 'tommy'
test_name_tensor = nameToTensor(test_name)

rnn_model.eval()
hiddne = rnn_model.get_hidden()
for char_idx in range(len(test_name)):
    char_tensor = test_name_tensor[char_idx]
    output, hidden = rnn_model(char_tensor[None,:],hidden)
predicted_index = torch.argmax(output, 1).item()
print(num2gen[predicted_index])


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]])
F
